![banner cnns ppgcc ufsc](http://www.lapix.ufsc.br/wp-content/uploads/2019/06/VC-lapix.png)

# Segmentation of Microfossils in Carbonatic Rocks

Notebook for the semantic segmentation of microfossil samples in MicroCT-acquired bore cores from carbonatic rocks. This notebook assumes that you are either using Google Colab or that you have the latest versions of PyTorch and fast.ai installed. You'll also need a GPU with at least 11 GB RAM. 


<a href="https://colab.research.google.com/drive/1jbP0mgesSVx7ibGucwFQdea709SARGhP"><img align="left"  src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>&nbsp; &nbsp;<a href=""><img align="left" src="http://www.lapix.ufsc.br/wp-content/uploads/2019/04/License-CC-BY-ND-4.0-orange.png" alt="Creative Commons 4.0 License" title="Creative Commons 4.0 License"></a>&nbsp; &nbsp; <a href=""><img align="left" src="http://www.lapix.ufsc.br/wp-content/uploads/2019/04/Jupyter-Notebook-v.1.0-blue.png" alt="Jupyter Version" title="Jupyter Version"></a>&nbsp; &nbsp;<a href=""><img align="left"  src="http://www.lapix.ufsc.br/wp-content/uploads/2019/04/Python-v.3.7-green.png" alt="Python Version" title="Python Version"></a> &nbsp; &nbsp;<a href=""><img align="left"  src="http://www.lapix.ufsc.br/wp-content/uploads/2019/04/fast.ai-v.1.0-red.png" alt="fastai Version" title="fastai Version"></a>

## Initializations, Import python libraries and fastai  Framework

In [ ]:
# Notebook Initializations
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Imports
from fastai.vision import *
from fastai.utils.show_install import *
from fastai.callbacks.hooks import *
from pathlib import Path
torch.backends.cudnn.benchmark=True

# Show if everything is OK
show_install()

## Define the places where your data is stored and check it

### If you are using Google Colab together with the Google Drive do this

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
path = Path('gdrive/My Drive/Colab Notebooks/DL/')
path.ls()

### If you are not importing your data from the Drive

In [ ]:
# Adapt this to match your environment...
path = Path('myPath/')
path.ls()

## Define data variables such as path_lbl (local where your labels are) and path_img (local where your train and validation datas are stored)

In [ ]:
# Initialize path_lbl (local where your labels are) 
path_lbl = path/'train_masks_labels'
# Initialize path_img (local where your train and validation data are stored)
path_img = path/'train'

In [ ]:
# Check how many files are there
fnames = get_image_files(path_img)
fnames[:3]
len(fnames)

In [ ]:
# Check if label names match the size
lbl_names = get_image_files(path_lbl)
lbl_names[:3]
len(lbl_names)

### Show a single MicroCT slice

In [ ]:
img_f = fnames[0]
img = open_image(img_f)
img.show(figsize=(5,5))

### Load the Mask belonging to this particular Slice

In [ ]:
# Scan the filenames with a simple lambda function
get_y_fn = lambda x: path_lbl/f'{x.stem}_GT{x.suffix}'

### Show a Ground Truth Mask

In [ ]:
mask = open_mask(get_y_fn(img_f))
mask.show(figsize=(5,5), alpha=1)
src_size = np.array(mask.shape[1:])

### Load your labels

In [ ]:
codes = np.loadtxt(path/'codes.txt', dtype=str); codes

## IOU metric, Initial data split and model 

In [ ]:
# Refer to your labels as numbers
name2id = {v:k for k,v in enumerate(codes)}

### Define your error metrics

The Intersection Over Union (IOU) Metric and a function to save prediction definitions

In [ ]:
def iou_metric(input, target):
    target = target.squeeze(1)
    mask = target != 0
    return (input.argmax(dim=1)[mask]==target[mask]).float().mean()

def save_preds(dl):
    i=0
    names = dl.dataset.items

    for b in dl:
      preds = learn.pred_batch(batch=b, reconstruct=True)
      for o in preds:
          o.save(path_gen/names[i].name)
          i += 1

### A few more definitions

Definition of weight decay, metric, model and if we employ imageNet weights

In [ ]:
wd=1e-2
metrics = iou_metric

# Use a deep network
used_model=models.resnet101

# We will employ transfer learning from ImageNet weights...
useImageNet=True

## Training Cycles & Validation

Where we start the training part. First we employ the 256x252(1/4) resolution. The same exact sequence is performed for the 512 (1/2) and 1024 (full) resolutions.

### Cycle \#1: 256x256

In [ ]:
# Define the batch size and the resolution employed

size = src_size//4
size[1]= size[1]+1
bs=5

Apply transformations such as resolution change and data augmentation to the data

In [ ]:
normalizePar=None

if useImageNet:
  normalizePar=imagenet_stats

  
data = (src.transform(get_transforms(), size=size, tfm_y=True)
        .databunch(bs=bs)
        .normalize(normalizePar))

Load the model with the data, the metric and the weight decay selected

In [ ]:
learn = unet_learner(data, used_model, metrics=metrics, wd=wd)

Find the most appropriate learning rate

In [ ]:
lr_find(learn)
learn.recorder.plot()

***Mannually*** set the learning rate after examining the graph above...

In [ ]:
# Adjust this LR accordingly to what you identified above...
lr=2e-4

#### Training strategy
- We will employ the  *fit1cycle* method developed by  Leslie N. Smith  - see below for details: 

  - https://docs.fast.ai/callbacks.one_cycle.html

  - A disciplined approach to neural network hyper-parameters: Part 1 -- learning rate, batch size, momentum, and weight decay - https://arxiv.org/abs/1803.09820
    
  - Super-Convergence: Very Fast Training of Residual Networks Using Large Learning Rates - https://arxiv.org/abs/1708.07120

- Since this method is *fast*, we will employ only 5 epochs in this first Transfer Learning stage

In [ ]:
learn.fit_one_cycle(5, slice(lr), pct_start=0.9)

Save the weitghs and load it to continue the trainning and perform some data release

In [ ]:
learn.save('stage1-256x252')

Keep a loading code in case you need it

In [ ]:
learn.load('stage1-256x252')

Unfreeze the learner in order to learn internal weights - Fine Tuning \@ Cycle #1

In [ ]:
learn.unfreeze()

Employ a Differential Learning Rate (DLR) and Train the learner

In [ ]:
lrs = slice(lr/400,lr/4)

In [ ]:
learn.fit_one_cycle(10, lrs, pct_start=0.8)

Save the weitghs and load it to continue the trainning and perform some data release

In [ ]:
# Save the fine-tuned network @ Cycle #1
learn.save('stage2-256x252')

In [ ]:
# Release Memory
del data
del learn
torch.cuda.empty_cache()
# Collect Garbage
gc.collect()

### Cycle \#2: 512x512

We will perform the same as above, just now with a resolution of 512x512

In [ ]:
# Set the new Size for the MicroCT Slices
size = src_size//2
bs=1

In [ ]:
# Adapt ImageNet Parameters to our Image Characteristics
normalizePar=None
if useImageNet:
    normalizePar=imagenet_stats
data = (src.transform(get_transforms(), size=size, tfm_y=True)
        .databunch(bs=bs)
        .normalize(normalizePar))

In [ ]:
# Create a new Network for 512x512 input images
learn = unet_learner(data, used_model, metrics=metrics, wd=wd)

In [ ]:
# Load our fine-tuned low resolution network weights learned on Cycle #1...
learn.load('stage2-256x252')

In [ ]:
# Find the best learning rate for this network instance
lr_find(learn)
learn.recorder.plot()

In [ ]:
# Manually set the new learning rate (LOOK at the graph above!)
lr=1e-3

Perform the transfer learning stage with this new, middle-resolution network. Again we will employ the  *fit1cycle* method developed by Leslie N. Smith.

In [ ]:
learn.fit_one_cycle(5, slice(lr), pct_start=0.8)

In [ ]:
# Save and Load...
learn.save('stage1-512x502')

In [ ]:
learn.load('stage1-512x502')

In [ ]:
# Unfreeze for fine-tuning...
learn.unfreeze()

Employ a Differential Learning Rate (DLR) and Train the learner

In [ ]:
# Prepare for varying learning rates...
lrs = slice(1e-6,lr/10)

In [ ]:
# Fine-tune for 10 epochs
learn.fit_one_cycle(10, lrs)

In [ ]:
# SAVE STAGE2 OF CYCLE #2...
learn.save('stage2-512x502')

In [ ]:
# Flush garbage..
del data
del learn
torch.cuda.empty_cache()
gc.collect()

### Cycle \#3: 1024x1024 - Full Resolution Training

Now we will perform the same as above again, just now with a resolution of 1024x1024. We will employ a shorter transfer learning stage.
Most cells below go uncommented because we are repeating steps, only with a few different parameters...

In [ ]:
# Original image size
size = src_size
# Batch size of one!
bs=1

In [ ]:
normalizePar=None
if useImageNet:
    normalizePar=imagenet_stats
data = (src.transform(get_transforms(), size=size, tfm_y=True)
        .databunch(bs=bs)
        .normalize(normalizePar))

In [ ]:
learn = unet_learner(data, used_model, metrics=metrics, wd=wd)

In [ ]:
learn.load('stage2-512x502')

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
# Adapt it to your values
lr=2e-5

In [ ]:
# Transfer learning stage
learn.fit_one_cycle(3, slice(lr), pct_start=0.8)

In [ ]:
# Save stage 1 of Cycle #3
learn.save('stage1-1024x1004')

In [ ]:
learn.load('stage1-1024x1004')

In [ ]:
# Prepare for the final fine-tuning
learn.unfreeze()

In [ ]:
# Prepare for varying learning rates
lrs = slice(1e-6,lr/10)

In [ ]:
# Fine-tune for 10 epochs
learn.fit_one_cycle(10, lrs)

In [ ]:
# Save stage 2 of Cycle #3
learn.save('stage2-1024x1004')

### Show some prediction results

In [ ]:
learn.show_results(rows=2, figsize=(10,10))

### Save all prediction results

In [ ]:
name_gen = 'image_gen'
path_gen = path/name_gen
path_gen.mkdir(exist_ok=True)

In [ ]:
save_preds(data.fix_dl)

![banner Creative Commons INCoD UFSC](http://www.lapix.ufsc.br/wp-content/uploads/2019/05/cc.png)